# 네이버 주식 종목토론방 크롤링 및 저장

본 코드는 다음의 사이트에서 참조를 하였습니다. 크롤링 세부 내용을 확인하려면 다음의 링크로 들어가시길 바라겠습니다.

출처 : 네이버 종목게시판 크롤링 / 디지털 노마드
링크 : https://lesrois.tistory.com/23

## 1. 필요 라이브러리 설치


In [1]:
#%pip install pandas-datareader
#!pip install -U finance-datareader
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
from datetime import datetime, timedelta#,strptime
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import re

import os 

plt.rcParams["figure.figsize"] = (12, 4)

## 2. 크롤링 코드

###   다음은 불러올 종목의 코드를 입력후, input_word에 원하는 검색어를 입력하면 원하는 단어가 포함된 게시글만 가져옵니다.
### 본 프로젝트에서는 전 게시글을 가져오므로 input_word를 빈칸으로 둡니다. 또한, 게시글이 많을경우 시간을 
### 고려해서 전체 게시물 수에 따라 넘기는 페이지 수가 다릅니다.

### 전체게시글 1000개 이상 -> 5페이지씩 게시글을 가져옵니다.
### 전체게시글 5000개 이상 -> 10페이지씩 게시글을 가져옵니다.
### 전체게시글 10000개 이상 -> 50페이지씩 게시글을 가져옵니다.
### 전체게시글 30000개 이상 -> 100페이지씩 게시글을 가져옵니다.

In [8]:
def roop_page(acodes="s",input_words="s"):
    total_dt = pd.DataFrame([])
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)'}
    input_word=input_words
    acode=acodes
    n_ = 0
    
                
    page=1

    url = "https://finance.naver.com/item/board.nhn?code=" + str(acode) + '&st=title&sw='+input_word+'&page='+ '100000'
    #print(url)
    #headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)'}
    result = requests.get(url, headers = headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    table = bs_obj.find('td', {'class' : 'on'})
    max_page=int(table.text.replace("\n","").replace(",",""))
    page_skip=1

    if max_page > 1000:
        page_skip=5

    if max_page > 5000:
        page_skip=10

    if max_page > 10000:
        page_skip=50

    if max_page > 30000:
        page_skip=100






    while True:
        n_ += page_skip
        if (n_ % 10 == 0):
            print('================== Page ' + str(page) + ' is done ==================')

        url = "https://finance.naver.com/item/board.nhn?code=" + str(acode) + '&st=title&sw='+input_word+'&page='+ str(page)
        result = requests.get(url, headers = headers)
        bs_obj = BeautifulSoup(result.content, "html.parser")
        table = bs_obj.find('table', {'class' : 'type2'})
        tt = table.select('tbody > tr')

        for i in range(2, len(tt)):
            if len(tt[i].select('td > span')) > 0:
                date = tt[i].select('td > span')[0].text
                title = tt[i].select('td.title > a')[0]['title']
                writer = tt[i].select('td.p11')[0].text.replace('\t', '').replace('\n', '')
                views = tt[i].select('td > span')[1].text
                pos = tt[i].select('td > strong')[0].text
                neg = tt[i].select('td > strong')[1].text

                table = pd.DataFrame({'날짜' : [date], 
                                    '제목' : [title],
                                    '글쓴이' : [writer],
                                    '조회' : [int(views)],
                                    '공감' : [int(pos)],
                                    '비공감' : [int(neg)]
                                    })
                total_dt = total_dt.append(table)
        page+=page_skip


        try:
            if len(total_dt_prev[(total_dt_prev["제목"]==title) & (total_dt_prev["날짜"]==date )].dropna())!=0  :
                print("전 게시글을 가져왔습니다.")
                break
            else:
                total_dt_prev=total_dt
        except:
            print("오류가 발생했습니다.")
            total_dt_prev=total_dt





    total_dt=total_dt.drop_duplicates()

    return total_dt


 

    
def naver_crawler(root,acode,search_keyword=""):
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)'}
    total_dt = pd.DataFrame([])
    page_skip=0
    
    Date=0

    Page=1
    
    input_word=search_keyword
    
    total_dt=roop_page(acodes=acode,input_words=input_word)
    
    total_dt=total_dt.drop_duplicates()
    return total_dt


# 유니버스 엑셀파일을 불러온뒤 데이터프레임화 과정입니다. 

In [10]:
universe_data=pd.read_excel("universe.xlsx").티커

for index,k in enumerate(universe_data):
    code_list=[]

    try:
        if len(str(int(k))) != 6:
            code=( 6 - len(str(int(k))) )*"0" + str(int(k))
            code_list.append(code)

        else:
            code_list.append(str(int(k)))

    except:
        continue

    if index==0:
        code_df=pd.DataFrame(code_list)
    else:
        code_df=pd.concat([code_df, pd.DataFrame(code_list) ],axis=1)
        
#code_df.columns=universe_data.columns
code_df

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,095570,068400,006840,027410,282330,138930,001460,001465,001040,079160,...,139990,271940,395400,329180,377300,381970,400760,402340,396690,404990


# 3. 유니버스 내의 모든 종목을 하나씩 for문으로 게시글을 크롤링해 csv파일로 저장하는 과정입니다.

In [11]:
root2=  'C:/Users/sosck/OneDrive/바탕 화면/stock_kosdaq_all_csv_now/'
for i in code_df.T.values:
    print(i[0])
    code=i[0]
    file1=root2 + code + "2021-12-31.csv"
    search_keyword="" 
    data=naver_crawler(root2,code,search_keyword="")
    data.to_csv(file1,encoding='utf-8-sig')

        


0
오류가 발생했습니다.
================== Page 10 is done ==================
================== Page 20 is done ==================


KeyboardInterrupt: 